In [5]:
import pandas as pd
import os
from PIL import Image
df = pd.read_csv("tableau_viz_png.csv", sep=";")
df

,viz_url,png_path,filename,folder_index,github_url
0,https://public.tableau.com/static/images/_4/_4...,viz_png\viz_0__4__4100.png,viz_0__4__4100.png,file_1/viz_0__4__4100.png,https://raw.githubusercontent.com/tomjarvisn/d...
1,https://public.tableau.com/static/images/18/18...,viz_png\viz_1_18_180315_class-culture_dotplots...,viz_1_18_180315_class-culture_dotplots.png,file_1/viz_1_18_180315_class-culture_dotplots.png,https://raw.githubusercontent.com/tomjarvisn/d...
2,https://public.tableau.com/static/images/Na/Na...,viz_png\viz_2_Na_NanosNational.png,viz_2_Na_NanosNational.png,file_1/viz_2_Na_NanosNational.png,https://raw.githubusercontent.com/tomjarvisn/d...
3,https://public.tableau.com/static/images/Vi/Vi...,viz_png\viz_3_Vi_VisualVocabulary.png,viz_3_Vi_VisualVocabulary.png,file_1/viz_3_Vi_VisualVocabulary.png,https://raw.githubusercontent.com/tomjarvisn/d...
4,https://public.tableau.com/static/images/To/To...,viz_png\viz_4_To_TopPlacestoRetire.png,viz_4_To_TopPlacestoRetire.png,file_1/viz_4_To_TopPlacestoRetire.png,https://raw.githubusercontent.com/tomjarvisn/d...
...,...,...,...,...,...
3171,https://public.tableau.com/static/images/GA/GA...,viz_png\viz_3171_GA_GAES19_1_4_1KeyTrends.png,viz_3171_GA_GAES19_1_4_1KeyTrends.png,file_7/viz_3171_GA_GAES19_1_4_1KeyTrends.png,https://raw.githubusercontent.com/tomjarvisn/d...
3172,https://public.tableau.com/static/images/Bo/Bo...,viz_png\viz_3172_Bo_BoeingCommercialMarketOutl...,viz_3172_Bo_BoeingCommercialMarketOutlook2018-...,file_7/viz_3172_Bo_BoeingCommercialMarketOutlo...,https://raw.githubusercontent.com/tomjarvisn/d...
3173,https://public.tableau.com/static/images/Av/Av...,viz_png\viz_3173_Av_AverageDomesticAirfareTime...,viz_3173_Av_AverageDomesticAirfareTimeSeries.png,file_7/viz_3173_Av_AverageDomesticAirfareTimeS...,https://raw.githubusercontent.com/tomjarvisn/d...
3174,https://public.tableau.com/static/images/IR/IR...,viz_png\viz_3174_IR_IRENAREsourceRenewableEner...,viz_3174_IR_IRENAREsourceRenewableEnergyCharts...,file_7/viz_3174_IR_IRENAREsourceRenewableEnerg...,https://raw.githubusercontent.com/tomjarvisn/d...


In [6]:
import pandas as pd
import os
from PIL import Image

# 1) Cargar el CSV original
df = pd.read_csv("tableau_viz_png.csv", sep=";")

# 2) Definimos las 7 carpetas nuevas
n_folders = 7
prefix_folder = "file_a_"  # así serán file_a_1, file_a_2, ..., file_a_7

# Creamos las carpetas (si no existen)
for i in range(1, n_folders + 1):
    carpeta = f"{prefix_folder}{i}"
    os.makedirs(carpeta, exist_ok=True)

# 3) Para cada fila, generar nombre “robusto” y mover/guardarlos en RGB
def sanitizar_nombre(idx):
    """
    Genera un nombre robusto basado en el índice:
    viz_0.png, viz_1.png, viz_2.png, ...
    """
    return f"viz_{idx}.png"

sanitized_paths = []

for idx, row in df.iterrows():
    # 3.1) Determinar a qué carpeta va (índice inicia en 0)
    folder_index = (idx // 500) + 1
    carpeta_dest = f"{prefix_folder}{folder_index}"

    # 3.2) Generar nombre nuevo robusto
    new_filename = sanitizar_nombre(idx)

    # 3.3) Leer el PNG original (asumiendo png_path con slash/backslash)
    #      Asegurate de que esa ruta sea correcta en tu repositorio local.
    original_path = row["folder_index"].replace("\\", "/")
    if not os.path.isfile(original_path):
        print(f"❌ No encontré este archivo: {original_path}")
        sanitized_paths.append("")  # dejamos vacío por seguridad
        continue

    # 3.4) Abrir con PIL, convertir a RGB (3 canales) y guardar optimizado
    img = Image.open(original_path)
    # Si venía en RGBA o P, convertir a RGB para evitar canal alfa problemática
    if img.mode in ("RGBA", "P", "LA"):
        img = img.convert("RGB")

    # Construir ruta destino
    destino = os.path.join(carpeta_dest, new_filename)

    # Guardar como PNG optimizado (sin canal alfa), así Tableau lo entienda
    img.save(destino, format="PNG", optimize=True)

    # 3.5) Anotar en lista para agregar al DataFrame
    sanitized_paths.append(destino)

# 4) Insertar la columna nueva en el DataFrame
df["sanitized_path"] = sanitized_paths

# 5) Guardar CSV final
df.to_csv("tableau_viz_png_sanitizado.csv", sep=";", index=False)

print("✅ Proceso finalizado. CSV guardado como 'tableau_viz_png_sanitizado.csv'.")


✅ Proceso finalizado. CSV guardado como 'tableau_viz_png_sanitizado.csv'.


In [13]:
df.to_csv("tableau_viz_png_sanitizado.csv", sep=";", index=False)


In [14]:
df

,viz_url,png_path,filename,folder_index,github_url,sanitized_path
0,https://public.tableau.com/static/images/_4/_4...,viz_png\viz_0__4__4100.png,viz_0__4__4100.png,file_1/viz_0__4__4100.png,https://raw.githubusercontent.com/tomjarvisn/d...,file_a_1\viz_0.png
1,https://public.tableau.com/static/images/18/18...,viz_png\viz_1_18_180315_class-culture_dotplots...,viz_1_18_180315_class-culture_dotplots.png,file_1/viz_1_18_180315_class-culture_dotplots.png,https://raw.githubusercontent.com/tomjarvisn/d...,file_a_1\viz_1.png
2,https://public.tableau.com/static/images/Na/Na...,viz_png\viz_2_Na_NanosNational.png,viz_2_Na_NanosNational.png,file_1/viz_2_Na_NanosNational.png,https://raw.githubusercontent.com/tomjarvisn/d...,file_a_1\viz_2.png
3,https://public.tableau.com/static/images/Vi/Vi...,viz_png\viz_3_Vi_VisualVocabulary.png,viz_3_Vi_VisualVocabulary.png,file_1/viz_3_Vi_VisualVocabulary.png,https://raw.githubusercontent.com/tomjarvisn/d...,file_a_1\viz_3.png
4,https://public.tableau.com/static/images/To/To...,viz_png\viz_4_To_TopPlacestoRetire.png,viz_4_To_TopPlacestoRetire.png,file_1/viz_4_To_TopPlacestoRetire.png,https://raw.githubusercontent.com/tomjarvisn/d...,file_a_1\viz_4.png
...,...,...,...,...,...,...
3171,https://public.tableau.com/static/images/GA/GA...,viz_png\viz_3171_GA_GAES19_1_4_1KeyTrends.png,viz_3171_GA_GAES19_1_4_1KeyTrends.png,file_7/viz_3171_GA_GAES19_1_4_1KeyTrends.png,https://raw.githubusercontent.com/tomjarvisn/d...,file_a_7\viz_3171.png
3172,https://public.tableau.com/static/images/Bo/Bo...,viz_png\viz_3172_Bo_BoeingCommercialMarketOutl...,viz_3172_Bo_BoeingCommercialMarketOutlook2018-...,file_7/viz_3172_Bo_BoeingCommercialMarketOutlo...,https://raw.githubusercontent.com/tomjarvisn/d...,file_a_7\viz_3172.png
3173,https://public.tableau.com/static/images/Av/Av...,viz_png\viz_3173_Av_AverageDomesticAirfareTime...,viz_3173_Av_AverageDomesticAirfareTimeSeries.png,file_7/viz_3173_Av_AverageDomesticAirfareTimeS...,https://raw.githubusercontent.com/tomjarvisn/d...,file_a_7\viz_3173.png
3174,https://public.tableau.com/static/images/IR/IR...,viz_png\viz_3174_IR_IRENAREsourceRenewableEner...,viz_3174_IR_IRENAREsourceRenewableEnergyCharts...,file_7/viz_3174_IR_IRENAREsourceRenewableEnerg...,https://raw.githubusercontent.com/tomjarvisn/d...,file_a_7\viz_3174.png


In [16]:


# Elegir las primeras 20 filas para el demo
demo_df = df.head(20)

# Guardar el nuevo CSV de demo
demo_df.to_csv("tableau_viz_png_demo.csv", index=False)

print("✅ Archivo demo 'tableau_viz_png_demo.csv' creado con 20 filas.")


✅ Archivo demo 'tableau_viz_png_demo.csv' creado con 20 filas.


In [20]:
import os
from PIL import Image
import pandas as pd

# Definir carpetas
carpetas = [f"file_a_{i}" for i in range(1, 8)]
archivos_convertidos = []

for carpeta in carpetas:
    for archivo in os.listdir(carpeta):
        if archivo.lower().endswith(".png"):
            ruta_png = os.path.join(carpeta, archivo)
            ruta_jpg = ruta_png.replace(".png", ".jpg")
            
            try:
                img = Image.open(ruta_png)
                if img.mode in ("RGBA", "P", "LA"):
                    img = img.convert("RGB")
                
                img.save(ruta_jpg, format="JPEG", quality=80, optimize=True)
                archivos_convertidos.append((ruta_png, ruta_jpg))
                
                # Podés comentar esta línea si querés conservar los PNG
                os.remove(ruta_png)
                
            except Exception as e:
                print(f"❌ Error al convertir {ruta_png}: {e}")

print(f"✅ {len(archivos_convertidos)} archivos convertidos a JPG")

# Actualizar CSV si tenés uno base
csv_path = "tableau_viz_png_sanitizado.csv"
df = pd.read_csv(csv_path, sep=";")

# Reemplazar .png por .jpg en las rutas que quedaron en sanitized_path
df["sanitized_path"] = df["sanitized_path"].str.replace(".png", ".jpg", regex=False)
df["github_url"] = df["github_url"].str.replace(".png", ".jpg", regex=False)

# Guardar CSV actualizado
df.to_csv("tableau_viz_jpg.csv", sep=";", index=False)
print("📁 CSV actualizado guardado como 'tableau_viz_jpg.csv'")


✅ 3176 archivos convertidos a JPG
📁 CSV actualizado guardado como 'tableau_viz_jpg.csv'
